<p style="font-family:'Times New Roman'; font-size:22px; color:Black;">
Introduction:
Heart attacks are a leading cause of death globally, emphasizing the need for early prediction to improve outcomes. This Project focuses on developing a machine learning model to predict heart attack risk using clinical data. Algorithms like Support Vector Machines (SVM), k-Nearest Neighbors (KNN), and Naïve Bayes were employed to classify patients based on features such as age, cholesterol,heart rate, diabetes, blood pressure etc. This project can help enhancing healthcare prediction.
</p>

In [23]:
# Import Libraries
import numpy as np

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



In [3]:
# import the datasets
data = pd.read_csv('heart_attack_prediction_dataset.csv')
pd.set_option('display.max_columns',None)
data.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Diet,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,0,4.168189,Average,0,0,9,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,1,1.813242,Unhealthy,1,0,1,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,0,2.078353,Healthy,1,1,9,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,1,9.828130,Average,1,0,9,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,0,5.804299,Unhealthy,1,0,6,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [4]:
# Find missing value
data.isnull().sum()

Patient ID                         0
Age                                0
Sex                                0
Cholesterol                        0
Blood Pressure                     0
Heart Rate                         0
Diabetes                           0
Family History                     0
Smoking                            0
Obesity                            0
Alcohol Consumption                0
Exercise Hours Per Week            0
Diet                               0
Previous Heart Problems            0
Medication Use                     0
Stress Level                       0
Sedentary Hours Per Day            0
Income                             0
BMI                                0
Triglycerides                      0
Physical Activity Days Per Week    0
Sleep Hours Per Day                0
Country                            0
Continent                          0
Hemisphere                         0
Heart Attack Risk                  0
dtype: int64

In [24]:
# Check duplicate data
print(data.duplicated().sum())

0


In [6]:
# shape of dataset
print('The Shape of our dataset is: ',data.shape)
data.dtypes

The Shape of our dataset is:  (8763, 26)


Patient ID                          object
Age                                  int64
Sex                                 object
Cholesterol                          int64
Blood Pressure                      object
Heart Rate                           int64
Diabetes                             int64
Family History                       int64
Smoking                              int64
Obesity                              int64
Alcohol Consumption                  int64
Exercise Hours Per Week            float64
Diet                                object
Previous Heart Problems              int64
Medication Use                       int64
Stress Level                         int64
Sedentary Hours Per Day            float64
Income                               int64
BMI                                float64
Triglycerides                        int64
Physical Activity Days Per Week      int64
Sleep Hours Per Day                  int64
Country                             object
Continent  

In [10]:
data=data[['Age', 'Sex', 'Cholesterol', 'Blood Pressure',
       'Heart Rate', 'Diabetes', 'Family History', 'Smoking', 'Obesity',
       'Alcohol Consumption', 'Exercise Hours Per Week',
       'Previous Heart Problems', 'Medication Use', 'Stress Level', 'BMI', 'Triglycerides','Heart Attack Risk'
       ]]

In [11]:
data.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Previous Heart Problems,Medication Use,Stress Level,BMI,Triglycerides,Heart Attack Risk
0,67,Male,208,158/88,72,0,0,1,0,0,4.168189,0,0,9,31.251233,286,0
1,21,Male,389,165/93,98,1,1,1,1,1,1.813242,1,0,1,27.194973,235,0
2,21,Female,324,174/99,72,1,0,0,0,0,2.078353,1,1,9,28.176571,587,0
3,84,Male,383,163/100,73,1,1,1,0,1,9.828130,1,0,9,36.464704,378,0
4,66,Male,318,91/88,93,1,1,1,1,0,5.804299,1,0,6,21.809144,231,0


In [12]:
# Separate blood pressure into 'Systolic' and 'Diastolic' values
data[['Systolic', 'Diastolic']] = data['Blood Pressure'].str.split('/', expand=True).astype(float)
data = data.drop(columns=['Blood Pressure'])

# Encode categorical variables
label_encoder = LabelEncoder()

data['Sex'] = label_encoder.fit_transform(data['Sex'])

# Define features and target
X = data.drop(columns=['Heart Attack Risk'])
y = data['Heart Attack Risk']

In [15]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

models=[SVC(),RandomForestClassifier(),GaussianNB(),KNeighborsClassifier()]
model_names=["Support Vector Machine","Gaussian Naive Bayes","K-Nearest Neighbors"]
models_scores=[]
for model,model_name in zip(models,model_names):
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    accuracy=accuracy_score(y_test,y_pred)
    models_scores.append([model_name,accuracy])
sorted_models_scores=sorted(models_scores,key=lambda x:x[1],reverse=True)
for model in sorted_models_scores:
    print("Accuracy Score: ",f'{model[0]}: {model[1]*100:.2f}')

Accuracy Score:  Support Vector Machine: 64.32
Accuracy Score:  K-Nearest Neighbors: 64.32
Accuracy Score:  Gaussian Naive Bayes: 62.91


In [26]:
# Train a Naive Bayes model

model=GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [27]:
# Predictions and Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


print("Accuracy of the model is: ",round(accuracy*100,2))

Accuracy of the model is:  64.32


In [28]:
scores=cross_val_score(model,X,y,cv=20)
print("Cross-Validation Accuracy: ",scores.mean()*100)


Cross-Validation Accuracy:  64.1789143029509
